In [9]:
import torch
import numpy as np

# Gradient Calculation

* Backward Propagation

In [20]:
# optimization을 위해 gradient를 계산하고 싶은 tensor에는 requires_grad=True를 설정해줘야 함. (default = False)
# pytorch에서 x에 대한 gradient를 tracking하기 위해 x.grad에 gradient를 저장함.
x = torch.randn(3, requires_grad=True)
print(x) 

y = x + 2 # 
print(y) # grad_fn=<AddBackward0>

z = y * y * 2
print(z) # grad_fn=<MulBackward0>

# vector Jacobian product(= chain rule) to get the gradient of the loss w.r.t. the input x
# so in this case since our z is a scalar, the output of this operation will be a vector of the same shape as x
z = z.mean() # (1, 3)을 (1, 1) = scalar로 만들어주기 위함. (1, 3)이었다면 Error.
print(z) # grad_fn=<MeanBackward0>

z.backward() # dz/dx

print(f"z.grad : {z.grad}")
print(f"y.grad : {y.grad}") 
print(f"x.grad : {x.grad}") # dz/dx = 2y = 2(x+2) = 2(1, 1, 1) = (2, 2, 2)

tensor([-0.9087,  0.7818,  0.1101], requires_grad=True)
tensor([1.0913, 2.7818, 2.1101], grad_fn=<AddBackward0>)
tensor([ 2.3821, 15.4772,  8.9046], grad_fn=<MulBackward0>)
tensor(8.9213, grad_fn=<MeanBackward0>)
z.grad : None
y.grad : None
x.grad : tensor([1.4551, 3.7091, 2.8134])


/home/hslee/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352660876/work/build/aten/src/ATen/core/TensorBody.h:477.)
  return self._grad


In [11]:
# optimization을 위해 gradient를 계산하고 싶은 tensor에는 requires_grad=True를 설정해줘야 함. (default = False)
# pytorch에서 x에 대한 gradient를 tracking하기 위해 x.grad에 gradient를 저장함.
x = torch.randn(3, requires_grad=True)
print(x) 

y = x + 2 
print(y) # grad_fn=<AddBackward0>

z = y * y * 2
print(z) # grad_fn=<MulBackward0>

# z = z.mean() # (1, 3) 

# z와 같은 shape의 tensor를 만들어줌.
v = torch.tensor([0.1, 1.0, 0.001], dtype=torch.float32) 

z.backward(v) 
print(x.grad) 

tensor([ 0.6201, -1.5607,  1.2418], requires_grad=True)
tensor([2.6201, 0.4393, 3.2418], grad_fn=<AddBackward0>)
tensor([13.7302,  0.3859, 21.0183], grad_fn=<MulBackward0>)
tensor([1.0481, 1.7570, 0.0130])


# Preventing Gradient History

* Three ways how we can stop pytorch from creating the gradient functions and tracking the history in our computational graph

## 1. x.requires_grad_(False)

In [12]:
x = torch.randn(3, requires_grad = True)
print(x)

x.requires_grad_(False) # in-place operation
print(x) # requires_grad = True가 이제 보이지 않음

tensor([ 0.0232, -0.3053, -0.6070], requires_grad=True)
tensor([ 0.0232, -0.3053, -0.6070])


## 2. x.detach()

In [13]:
x = torch.randn(3, requires_grad = True)
print(x)

x.detach() # requires_grad = False인 tensor를 return함.
print(x) 

y  = x.detach()
print(y)

tensor([-0.1252,  0.7873, -2.5480], requires_grad=True)
tensor([-0.1252,  0.7873, -2.5480], requires_grad=True)
tensor([-0.1252,  0.7873, -2.5480])


## 3. with torch.no_grad() :

In [14]:
x = torch.randn(3, requires_grad = True)
print(x)

y = x + 2
print(y)

with torch.no_grad() : # with문 안에서는 gradient를 계산하지 않음.
    y = x + 2
    print(y) # grad_fn=<AddBackward0>이 여기서는 나오지 않음.


tensor([ 0.0767, -0.4190, -0.4595], requires_grad=True)
tensor([2.0767, 1.5810, 1.5405], grad_fn=<AddBackward0>)
tensor([2.0767, 1.5810, 1.5405])


# .backward() 주의사항 : (acuumulated grad)

In [15]:
weights = torch.ones(4, requires_grad=True)

# .backward() will accumulate gradients into .grad instead of overwriting
# pytorch에서는 gradient를 더해주는 것이 default임.
# 그래서 정확한 gradient를 얻지 못하게 된다.
for epoch in range(3) :
    model_output = (weights*3).sum()
    model_output.backward()
    print(weights.grad) 

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])


In [16]:
weights = torch.ones(4, requires_grad=True)

# 따라서 정확한 gradient를 얻기 위해서는 .backward()를 호출할 때마다 .grad를 0으로 초기화해줘야 함.
for epoch in range(3) :
    model_output = (weights*3).sum()
    model_output.backward()
    print(weights.grad) 
    
    weights.grad.zero_() # gradient를 0으로 초기화해줘야 함.

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
